## Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import networkx 
from networkx.algorithms.components.connected import connected_components

## Import Dataset

In [2]:
# Dataset can be accessed in https://www.kaggle.com/c/scl-2021-da/overview
df = pd.read_json('contacts.json')

In [3]:
df.head(10)

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,
5,5,,840113148,0,
6,6,hESiBgYTxMnj@hotmail.com,,4,
7,7,hqyunTiaXfIjhO@yahoo.com,,1,vxjYZtMUvNhtEBDruzQfjpsmX
8,8,XGUjcSiDpFKWEUCs@qq.com,223604620644,2,
9,9,osIIQgAiZX@hotmail.com,,0,IqewrMXVTJqSsVklWFfOpdRNI


### We can see that the dataset is full of '', so our first task is to replace the '' to Nan

In [4]:
df = df.replace('', np.nan)

In [5]:
df.head(10)

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,NaN,1,NaN
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,NaN,9125983679,0,NaN
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,NaN,300364407,2,NaN
5,5,NaN,840113148,0,NaN
6,6,hESiBgYTxMnj@hotmail.com,NaN,4,NaN
7,7,hqyunTiaXfIjhO@yahoo.com,NaN,1,vxjYZtMUvNhtEBDruzQfjpsmX
8,8,XGUjcSiDpFKWEUCs@qq.com,223604620644,2,NaN
9,9,osIIQgAiZX@hotmail.com,NaN,0,IqewrMXVTJqSsVklWFfOpdRNI


## Problem Formulation and Problem Solving Plan

Next, to solve this multi-channel contacts problem, my general flow is basically:

1.  Find and gather "ID" that has common element in each column.                                    This way, you will a list of "ID" that shares same Order ID, Email, Phone
2.  As explained in the problem overview, some ticket also linked cross-column.                     We need to check for every list of "ID" we got from Step 1, whether they share at              least 1 "ID" with others or not. If it is yes, then we need to combine it.
 3.  After we get the final list of "ID", all that's left is to count the sum of the                 contacts. 


![title](Intro-1.jpg)

![title](Intro-2.jpg)

## 1. Link Ticket with Common Element in Each Column

In [6]:
# Simplest - but not the fastest - way is to use groupby and apply set
# Don't forget to use subset notNa because we don't need row with NaN entry
common_email = df[df['Email'].notna()].groupby('Email')['Id'].apply(set)
common_phone = df[df['Phone'].notna()].groupby('Phone')['Id'].apply(set)
common_orderid = df[df['OrderId'].notna()].groupby('OrderId')['Id'].apply(set)

In [7]:
# Example: "AAAQbNfGbLAeOvw@hotmail.com" email has 2 ID members 
common_email

Email
AAAQbNfGbLAeOvw@hotmail.com         {470075, 112387}
AAAgyKTteOjasN@qq.com                       {338154}
AABZmKeEldvO@gmail.com                      {426495}
AABneiGcqcz@qq.com                           {37868}
AABtOIZmdYdKGnidAK@yahoo.com                {137956}
                                          ...       
zzyYjbPWM@hotmail.com               {143089, 338473}
zzyacSRLb@gmail.com                         {425119}
zzylmjkoqfCkTOxi@hotmail.com                {146556}
zzyvKfCcQWPJdShGoItP@hotmail.com    {381064, 458030}
zzyvegdhAJTFNDRQhMS@hotmail.com             {232950}
Name: Id, Length: 249156, dtype: object

In [8]:
# Next, we have to concatenate all the list of ID into one long list of IDs
common_concat = pd.concat([common_email, common_phone, common_orderid])

## 2. Link Ticket with Cross Column Link

Simplest way to this is to use Networkx Library to make graph.
Imagine each ID you gathered before is graph nodes, and you want to find all the connections for all the graph nodes

In [9]:
def to_graph(l):
    G = networkx.Graph()
    for part in l:
        # For every sublist, imagine it as list of graph nodes
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current    

graph = to_graph(common_concat)

In [10]:
# Extract the connected components to a list
common_all = [list(i) for i in connected_components(graph)]

In [11]:
# We have connected every ID that's available. You can see below
common_all[0:10]

[[112387, 470075],
 [338154],
 [426495],
 [308856, 37868, 126317],
 [137956],
 [78833, 407938, 494574],
 [19504, 179125],
 [73472, 310113, 216393, 482411, 475822, 435438, 299505, 51764],
 [299171],
 [386346]]

## 3. Start Counting the Contacts

In [12]:
# Next, for every sublist, then for every ID in each sublist, sum the contacts

sum_count = []

for row in common_all:
    count = 0
    for i in row:
        count += df['Contacts'][i]
    sum_count.append(count)

In [13]:
# Formatting
# Next, for every sublist, then for every ID in each sublist, we need to reformat the text to follow Shopee's instructions

ticket_trace = []

for row in common_all:
    row = sorted(row)
    formatted = [str(i) for i in row]
    formatted = '-'.join(formatted)
    ticket_trace.append(formatted)

In [14]:
# Combine the original list of ID, ticket trace (list of ID but reformatted), and sum of counts
combined_list = list(zip(common_all, ticket_trace, sum_count))

In [15]:
# You can see the result like this
combined_list[0:10]

[([112387, 470075], '112387-470075', 4),
 ([338154], '338154', 2),
 ([426495], '426495', 2),
 ([308856, 37868, 126317], '37868-126317-308856', 7),
 ([137956], '137956', 1),
 ([78833, 407938, 494574], '78833-407938-494574', 6),
 ([19504, 179125], '19504-179125', 3),
 ([73472, 310113, 216393, 482411, 475822, 435438, 299505, 51764],
  '51764-73472-216393-299505-310113-435438-475822-482411',
  21),
 ([299171], '299171', 0),
 ([386346], '386346', 4)]

In [16]:
# Next, we need to "explode" the combined list to become the original 500.000 rows DataFrame, complete with ticket trace and sum contact

exploded = []

for row in combined_list:
    for i in row[0]:
        ticket_n_contact = row[1] + ", " + str(row[2])
        exploded.append([i, ticket_n_contact])

exploded = sorted(exploded, key = lambda x: x[0])

In [17]:
# Convert to DataFrame for Final Result

df_result = pd.DataFrame(exploded, columns=["ticket_id", "ticket_trace/contact"])
df_result

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
499995,499995,"499995, 2"
499996,499996,"499996, 4"
499997,499997,"499997, 2"
499998,499998,"121111-499998, 5"
